In [262]:
puzzle = open('puzzle.txt', 'r').read()
example = open('example.txt', 'r').read()
test = open('test.txt', 'r').read()

In [263]:
input = puzzle
fresh_ranges, ingredients = input.split('\n\n')
fresh_ranges = fresh_ranges.split('\n')
ingredients = ingredients.split('\n')

# Part 1

In [264]:
fresh_count = 0

for ingredient in ingredients:
    for fresh_range in fresh_ranges:
        lower,upper = fresh_range.split('-')
        if int(ingredient) <= int(upper) and int(ingredient) >= int(lower):
            fresh_count += 1
            break

fresh_count

726

# Part 2

In [265]:
non_overlapping_ranges = set()

for fresh_range in fresh_ranges:
    cur_lower,cur_upper = map(int, fresh_range.split('-'))
    #print(f'\nHashset: {sorted(non_overlapping_ranges)}')
    #print(f'Range to insert: {cur_lower}-{cur_upper}')

    overlaps = False
    lower_overlap = upper_overlap = False
    lower_overlap_existing_range = upper_overlap_existing_range = None
    existing_encompassed = set()
    for non_overlapping_range in non_overlapping_ranges:
        lower_boundary_clips = upper_boundary_clips = False
        non_overlapping_ranges_lower, non_overlapping_ranges_upper = non_overlapping_range[0],non_overlapping_range[1]
        # [] => range to insert (cur_lower, cur_upper)
        # () => exisiting range (non_overlapping_ranges_lower, non_overlapping_ranges_upper)

        # Is the lower boundary clipping?
        if cur_lower <= non_overlapping_ranges_upper and cur_lower >= non_overlapping_ranges_lower:
            #print('Lower boundary clips')
            lower_boundary_clips = True

        # Is the upper boundary clipping?
        if cur_upper >= non_overlapping_ranges_lower and cur_upper <= non_overlapping_ranges_upper:
            #print('Upper boundary clips')
            upper_boundary_clips = True

        # ([]) => range to insert already included. Do nothing & skip to next range to insert

        # ([)] => lower boundary clipped. Move the existing upper boundary to the new upper boundary
        if lower_boundary_clips:
            lower_overlap_existing_range = non_overlapping_range
            lower_overlap = True
        
        # [(]) => upper boundary clipped. Move the existing lower boundary to the new lower boundary
        if upper_boundary_clips:
            upper_overlap_existing_range = non_overlapping_range
            upper_overlap = True

        if lower_boundary_clips or upper_boundary_clips:
            continue

        # [()] => Existing is entirely contained within new range. Remove this range but keep going as we need to check for any others
        if cur_lower <= non_overlapping_ranges_lower and cur_upper >= non_overlapping_ranges_upper:
            existing_encompassed.add((non_overlapping_ranges_lower,non_overlapping_ranges_upper))

        # ()[] or []() => no overlaps. Continue to check other existing ranges. If no overlaps with any, add new range to existing ones as it is.

        # ([)(]) => Remove two existing ranges and insert a new range with the first's lower boundary and second's upper boundary

    
    # Remove the () in the [()] cases
    for range_to_remove in existing_encompassed:
        non_overlapping_ranges.remove(range_to_remove)

    # Resolve ([)(]) case to ()
    if lower_overlap and upper_overlap and (lower_overlap_existing_range != upper_overlap_existing_range):
        #print(f'CHANGING {lower_overlap_existing_range} AND {upper_overlap_existing_range} TO {(lower_overlap_existing_range[0], upper_overlap_existing_range[1])}')
        non_overlapping_ranges.remove(lower_overlap_existing_range)
        non_overlapping_ranges.remove(upper_overlap_existing_range)
        non_overlapping_ranges.add((lower_overlap_existing_range[0], upper_overlap_existing_range[1]))
    
    # Resolve ([)] to (]
    elif lower_overlap and not upper_overlap:
        #print(f'CHANGING {lower_overlap_existing_range} TO {(lower_overlap_existing_range[0], cur_upper)}')
        non_overlapping_ranges.remove(lower_overlap_existing_range)
        non_overlapping_ranges.add((lower_overlap_existing_range[0], cur_upper))

    # Resolve [(]) to [)
    elif upper_overlap and not lower_overlap:
        #print(f'CHANGING {upper_overlap_existing_range} TO {(cur_lower, upper_overlap_existing_range[1])}')
        non_overlapping_ranges.remove(upper_overlap_existing_range)
        non_overlapping_ranges.add((cur_lower,upper_overlap_existing_range[1]))

    if not (lower_overlap or upper_overlap):
        non_overlapping_ranges.add((cur_lower,cur_upper))
    
    #print(f'Changed Hashset: {sorted(non_overlapping_ranges)}')
        

#print(f'Final ranges: {sorted(non_overlapping_ranges)}')

In [266]:
total_count = 0

for non_overlapping_range in sorted(non_overlapping_ranges):
    lower, upper = non_overlapping_range[0], non_overlapping_range[1]

    # +1 accounts for inclusivity of range boundaries
    total_count += (upper-lower)+1
    #print(f'Range: {non_overlapping_range}')
    #print(f'value: {(upper-lower)+1}')

total_count

354226555270043